In [4]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Eindopdracht Inleiding Vision
# Tom Schoonbeek 2032257 & Djim Oomes 2122380

# Webcam
cam = cv2.VideoCapture(0)

# Capture Window
cv2.namedWindow("Bottle Inspection")

img_counter = 0

# Running loop
while True:
    ret, frame = cam.read()
    if not ret:
        print("Fout bij verbinden met camera.")
        break
    cv2.putText(frame, "Druk op spatie om fles te analyseren", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
    cv2.imshow("Bottle Inspection", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC button press
        print("ESC ingedrukt. Sluit applicatie.")
        print("<end>")
        print("")
        break
    elif k%256 == 32:
        # SPACE button press
        img_name = "fles_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} vastgelegd en geschreven naar bestand".format(img_name))
        
        # Foto inladen die net is opgeslagen
        #bottle_3_channel = cv2.imread(img_name)
        
        # Sample images als alternatief
        bottle_3_channel = cv2.imread('fles_sample_correct.png')
        #bottle_3_channel = cv2.imread('fles_sample_high.png')
        #bottle_3_channel = cv2.imread('fles_sample_low.png')
        #bottle_3_channel = cv2.imread('fles_sample_nolid.png')
        
        bottle_gray = cv2.split(bottle_3_channel)[0]
        
        # Checkmarkers voor beide checks later in de code
        bottle_contents_check = 0;
        bottle_cap_check = 0;
        
        # Gaussian Blur
        bottle_gray = cv2.GaussianBlur(bottle_gray, (3, 3), 0)
        print('Gaussian Blur toegepast om de image te blurren en ruis te verminderen.')
        
        # Manual threshold + contour fles
        threshold_value = 198.5
        kernel = np.ones((5,5),np.uint8)
        (T, bottle_threshold) = cv2.threshold(bottle_gray, threshold_value, 255, cv2.THRESH_BINARY_INV)
        contour_bottle = cv2.morphologyEx(bottle_threshold,cv2.MORPH_GRADIENT,kernel)
        print('Thresholding toegepast voor een duidelijke splitsing van hoge en lage waarden van de image d.m.v. zwarte en witte pixels.')
        cv2.imshow("Contour fles", contour_bottle)
        cv2.waitKey(0)
        
        # Invert image voor Erosion & Opening
        bottle_threshold = cv2.bitwise_not(bottle_threshold)
        
        # Erosion
        kernel = np.ones((3,3),np.uint8)
        bottle_eroded = cv2.erode(bottle_threshold,kernel,iterations = 3)
        print('Erosion toegepast om delen van de image van elkaar te scheiden.')
        
        # Opening (Erosion gevolgd door Dilation) - Hierna is inhoud fles en flesdop normaalgesproken geïsoleerd
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        opening_internal = cv2.morphologyEx(bottle_eroded, cv2.MORPH_OPEN, kernel)
        print('Opening toegepast. Wederom een dosis Erosion + een stukje Dilation (weghalen ruis)')
        
        # Invert image terug naar origineel
        opening_internal = cv2.bitwise_not(opening_internal)
        
        # Contours
        image, contours, hierarchy = cv2.findContours(opening_internal, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        print('Contours berekend.')
        
        # Internal contours (vloeistof+flesdop)
        internal_contours = np.zeros(image.shape)
        for i in range(len(contours)):
            if hierarchy[0][i][3] != -1: # checkt laatste value uit elke row van de contours array op value die niet -1 is
                cv2.drawContours(internal_contours, contours, i, 255, -1)
        # Sorteren per grootte van area, pakt de eennagrootste waarde voor de flesinhoud (grootste waarde is de achtergrond) en de tweenagrootste voor flesdop
        areas = [cv2.contourArea(contour) for contour in contours]
        (contours, areas) = zip(*sorted(zip(contours, areas), key=lambda a:a[1]))
        
        # Contour Inhoud fles
        i = len(contours)-2
        bottle_clone = bottle_3_channel.copy()
        cv2.drawContours(bottle_clone, [contours[i]], -1, (255, 0, 0), 2)
        cv2.imshow("Contour flesinhoud", bottle_clone)
        cv2.waitKey(0)
        
        # Contour Flesdop
        j = len(contours)-3
        bottle_clone = bottle_3_channel.copy()
        cv2.drawContours(bottle_clone, [contours[j]], -1, (255, 0, 0), 2)
        cv2.imshow("Contour flesdop", bottle_clone)
        cv2.waitKey(0)
        
        # Bounding box maken voor berekenen aspect ratio van inhoud fles en goed- en afkeuren
        print('Check inhoud fles op 80-90% inhoud..')
        bottle_clone = bottle_3_channel.copy()
        (x, y, w, h) = cv2.boundingRect(contours[i]) #i vanwege inhoud fles
        aspectRatio = w / float(h)
        
        # Vulpercentage berekend in Excel-sheet
        if aspectRatio < 0.380:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te hoog", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "100%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            print('--FAIL!')
        elif aspectRatio > 0.380 and aspectRatio < 0.421940928:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te hoog", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "90-100%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            print('--FAIL!')
        elif aspectRatio > 0.421940928 and aspectRatio < 0.474683544:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(bottle_clone, "Correct", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
            cv2.putText(bottle_clone, "80-90%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
            print('--PASS!')
            bottle_contents_check = 1;
        elif aspectRatio > 0.474683544 and aspectRatio < 0.542495479:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te laag", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "70-80%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            print('--FAIL!')
        elif aspectRatio > 0.542495479 and aspectRatio < 0.632911392:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te laag", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "60-70%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            print('--FAIL!')
        elif aspectRatio > 0.632911392 and aspectRatio < 0.759493671:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te laag", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "50-60%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            print('--FAIL!')
        elif aspectRatio > 0.759493671:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te laag", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "<50%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            print('--FAIL!')
        cv2.imshow("Controle flesinhoud", bottle_clone)
        cv2.waitKey(0)
        
        # Check  flesdop
        print('Check flesdop op aanwezigheid en correctheid..')
        # Initiate SIFT detector
        sift = cv2.xfeatures2d.SIFT_create()
        
        # Source + target
        source = cv2.imread('bottle_cap_sample.png', 0)     
        target = bottle_3_channel.copy()   

        # find the keypoints and descriptors with SIFT
        kp1, des1 = sift.detectAndCompute(source,None)
        kp2, des2 = sift.detectAndCompute(target,None)

        # FLANN parameters
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)  

        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        good = []

        # Ratio test
        for i,(match1,match2) in enumerate(matches):
            if match1.distance < 0.3*match2.distance:
                good.append([match1])
        flann_matches = cv2.drawMatchesKnn(source,kp1,target,kp2,good,None,flags=0)

        # Resultaat flesdop check
        if not good:
            #display(no_match)
            cv2.putText(flann_matches, "Dop error", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            print('--FAIL!')
            cv2.imshow("Controle flesdop", flann_matches)
            cv2.waitKey(0)
        else:
            #display(flann_matches)
            cv2.putText(flann_matches, "Correct", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
            print('--PASS!')
            cv2.imshow("Controle flesdop", flann_matches)
            bottle_cap_check = 1;
            cv2.waitKey(0)
            
        # Final keuring
        if bottle_contents_check == 1 and bottle_contents_check == 1:
            approved_img = cv2.imread('quality_approved.jpg',1)
            print('Alle checks PASSED! Fles goedgekeurd.')
            print('')
            cv2.imshow("Fles goedgekeurd", approved_img)
            cv2.waitKey(0)
        else:
            rejected_img = cv2.imread('quality_rejected.jpg',1)
            print('Eén of meerdere checks FAILED! Fles afgekeurd.')
            print('')
            cv2.imshow("Fles afgekeurd", rejected_img)
            cv2.waitKey(0)
        
        ## untested, nog checken:
        # Reset van de checkwaardes voor volgende iteratie van loop
        bottle_cap_check = 0;
        bottle_contents_check = 0;
        
cam.release()

cv2.destroyAllWindows()

fles_0.png vastgelegd en geschreven naar bestand
Gaussian Blur toegepast om de image te blurren en ruis te verminderen.
Thresholding toegepast voor een duidelijke splitsing van hoge en lage waarden van de image d.m.v. zwarte en witte pixels.
Erosion toegepast om delen van de image van elkaar te scheiden.
Opening toegepast. Wederom een dosis Erosion + een stukje Dilation (weghalen ruis)
Contours berekend.
Check inhoud fles op 80-90% inhoud..
--PASS!
Check flesdop op aanwezigheid en correctheid..
--PASS!
Alle checks PASSED! Fles goedgekeurd.

ESC ingedrukt. Sluit applicatie.
<end>

